In [1]:
import pandas as pd
import numpy as np
from math import log2
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as mticker
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
from statsmodels.stats.inter_rater import fleiss_kappa
from statsmodels.stats import inter_rater as irr
from statsmodels.tsa.stattools import adfuller
import pingouin as pg
pd.options.mode.chained_assignment = None

In [2]:
def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]

C:\Users\succa\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
C:\Users\succa\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
def get_Fame(arr):
    first_entry = np.where(arr != 0)[0][0]
    return np.median(arr[first_entry:])

In [4]:
def get_Fame_ave(arr):
    first_entry = np.where(arr != 0)[0][0]
    return np.mean(arr[first_entry:])

In [5]:
def cronbach_alpha(df):
    df_corr = df.corr()
    N = df.shape[1]
    rs = np.array([])
    for i, col in enumerate(df_corr.columns):
        sum_ = df_corr[col][i+1:].values
        rs = np.append(sum_, rs)
    mean_r = np.mean(rs)
    
    cronbach_alpha = (N * mean_r) / (1 + (N - 1) * mean_r)
    return cronbach_alpha

In [6]:
Mass_shooting_df = pd.read_excel("Mass_Shootings_dummy.xlsx", sheet_name = "Mass_Shootings").iloc[0:10]
Mass_shooting_df = Mass_shooting_df.drop_duplicates(subset=['Date'])
Mass_shooting_df = Mass_shooting_df.reset_index()
N = len(Mass_shooting_df)
Variables = ["Target_Group","Shooting_Location","Level_of_Security","Leakage"]
wo = np.zeros([N,N])
for i in range(N):
    for j in range(i):
        wo[i,j] = 1
for Variable in Variables:
    Mass_shooting_df = Mass_shooting_df.reindex(columns = Mass_shooting_df.columns.tolist() + [Variable+"_Surprisal"])
    
    

for index , Variable in enumerate(Variables):
    uniques = np.unique(Mass_shooting_df[Variable])
    L = len(uniques)
    C = np.zeros([N, int(L)])
    J = np.ones([N, int(L)])
    for i in range(N):
        for j in range(len(uniques)):
            C[i, j] = 1 if (Mass_shooting_df[Variable][i] == uniques[j]) else 0
    priorDis = np.dot(wo,C)
    choices = np.argwhere(C==1)[:,1]
    S = np.array([])
    x = np.array([])
    colors = np.array([])
    fs = np.array([])
    nfs = np.array([])
    for i in range(1,N):
        history = priorDis[i,:]
        choice = priorDis[i,choices[i]]
        y = -log2((choice+1)/(L+np.sum(history)))
        Mass_shooting_df[Variable+"_Surprisal"][i] = y
Mass_shooting_df = Mass_shooting_df.iloc[1:]
Mass_shooting_df = Mass_shooting_df.reset_index()

In [7]:
#Mass_shooting_df=Mass_shooting_df.iloc[49:].reset_index(drop=True) ## use if you want to igonre first couple of shootings 
for Variable in Variables:
    fs = np.array(Mass_shooting_df.loc[Mass_shooting_df["FameSeeker_Silva"]][Variable+"_Surprisal"])
    nfs = np.array(Mass_shooting_df.loc[Mass_shooting_df["FameSeeker_Silva"]==False][Variable+"_Surprisal"])
    pval = mannwhitneyu(nfs, fs, alternative='less',method='auto')
    print(Variable,pval)

Target_Group MannwhitneyuResult(statistic=9.0, pvalue=0.4504326946603251)
Shooting_Location MannwhitneyuResult(statistic=16.0, pvalue=0.9444444444444445)
Level_of_Security MannwhitneyuResult(statistic=0.0, pvalue=0.007539249217821244)
Leakage MannwhitneyuResult(statistic=14.5, pvalue=0.8906289949945501)


In [9]:
fame_seeking_df = Mass_shooting_df[(Mass_shooting_df["FameSeeker_Silva"]==True)]
Extended = fame_seeking_df[(fame_seeking_df["Used an Extended Magazine"]==1.0)]
NotExtended = fame_seeking_df[(fame_seeking_df["Used an Extended Magazine"]==0.0)]
print(mannwhitneyu(Extended["Target_Group_Surprisal"], NotExtended["Target_Group_Surprisal"], method='auto'))
print(mannwhitneyu(Extended["Shooting_Location_Surprisal"], NotExtended["Shooting_Location_Surprisal"], method='auto'))
print(mannwhitneyu(Extended["Level_of_Security_Surprisal"], NotExtended["Level_of_Security_Surprisal"], method='auto'))
print(mannwhitneyu(Extended["Leakage_Surprisal"], NotExtended["Leakage_Surprisal"], method='auto'))

MannwhitneyuResult(statistic=4.0, pvalue=0.22067136191984682)
MannwhitneyuResult(statistic=0.0, pvalue=0.3333333333333333)
MannwhitneyuResult(statistic=0.0, pvalue=0.3333333333333333)
MannwhitneyuResult(statistic=4.0, pvalue=0.3333333333333333)


In [10]:
Wikipedia_df = pd.read_excel("Mass_Shootings_dummy.xlsx", sheet_name = "Wikiepdia")
for col in Wikipedia_df.columns:
    if col != "DateTime":
        Wikipedia_df[col] = pd.to_numeric(Wikipedia_df[col])
Wikipedia_df["DateTime"] = pd.to_datetime(Wikipedia_df["DateTime"])

In [11]:
Mass_shooting_df["Fame"] = np.zeros([len(Mass_shooting_df)])
Mass_shooting_df["Fame_av"] = np.zeros([len(Mass_shooting_df)])

for i in range(len(Mass_shooting_df)):
    if (Mass_shooting_df["WikiPage_reference"][i]=="There's a wikipedia page but not on the data site") | (Mass_shooting_df["WikiPage_reference"][i]=="No_page") | (Mass_shooting_df["WikiPage_reference"][i]=="Corrupted"): 
        Mass_shooting_df["Fame"][i] = np.nan
        Mass_shooting_df["Fame_av"][i] = np.nan
    else:
        fame_score = get_Fame(Wikipedia_df[Mass_shooting_df["WikiPage_reference"][i]].values)
        Mass_shooting_df["Fame"][i] = fame_score
        fame_score_av = get_Fame_ave(Wikipedia_df[Mass_shooting_df["WikiPage_reference"][i]].values)
        Mass_shooting_df["Fame_av"][i] = fame_score_av

In [12]:
Correlation_df = Mass_shooting_df.dropna(subset=['Fame'])[["Fame","Fame_av","Injured","Fatalities","Target_Group_Surprisal",
                                                          "Shooting_Location_Surprisal","Level_of_Security_Surprisal",
                                                          "Leakage_Surprisal"]]

In [13]:
display(pg.partial_corr(data=Correlation_df, x='Fame', y='Target_Group_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame', y='Shooting_Location_Surprisal', covar=['Fatalities','Injured'], method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame', y='Level_of_Security_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame', y='Leakage_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))

,n,r,CI95%,p-val
spearman,9,0.1437,"[-0.68, 0.81]",0.7585


,n,r,CI95%,p-val
spearman,9,-0.5882,"[-0.93, 0.3]",0.1648


,n,r,CI95%,p-val
spearman,9,0.1193,"[-0.7, 0.8]",0.7989


,n,r,CI95%,p-val
spearman,9,0.2522,"[-0.62, 0.84]",0.5853


In [14]:
display(pg.partial_corr(data=Correlation_df, x='Fame_av', y='Target_Group_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame_av', y='Shooting_Location_Surprisal', covar=['Fatalities','Injured'], method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame_av', y='Level_of_Security_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))
display(pg.partial_corr(data=Correlation_df, x='Fame_av', y='Leakage_Surprisal', covar=['Fatalities','Injured'],method = "spearman").round(4))

,n,r,CI95%,p-val
spearman,9,0.0822,"[-0.72, 0.79]",0.8608


,n,r,CI95%,p-val
spearman,9,-0.6944,"[-0.95, 0.12]",0.0834


,n,r,CI95%,p-val
spearman,9,0.2636,"[-0.61, 0.85]",0.5678


,n,r,CI95%,p-val
spearman,9,0.5097,"[-0.4, 0.91]",0.2426


In [15]:
Correlation_df.corr(method='spearman')

,Fame,Fame_av,Injured,Fatalities,Target_Group_Surprisal,Shooting_Location_Surprisal,Level_of_Security_Surprisal,Leakage_Surprisal
Fame,1.000000,0.900000,0.369761,0.263523,0.186467,-0.150000,-0.069631,0.343099
Fame_av,0.900000,1.000000,0.260513,0.430421,0.254274,-0.200000,0.139262,0.443519
Injured,0.369761,0.260513,1.000000,0.013287,-0.170946,0.470605,-0.618802,0.518992
Fatalities,0.263523,0.430421,0.013287,1.000000,0.562859,0.377716,0.311940,-0.141135
Target_Group_Surprisal,0.186467,0.254274,-0.170946,0.562859,1.000000,0.033903,0.035411,-0.042557
Shooting_Location_Surprisal,-0.150000,-0.200000,0.470605,0.377716,0.033903,1.000000,-0.295932,0.000000
Level_of_Security_Surprisal,-0.069631,0.139262,-0.618802,0.311940,0.035411,-0.295932,1.000000,-0.327764
Leakage_Surprisal,0.343099,0.443519,0.518992,-0.141135,-0.042557,0.000000,-0.327764,1.000000


In [16]:
Correlation_df.corr(method=spearmanr_pval)

,Fame,Fame_av,Injured,Fatalities,Target_Group_Surprisal,Shooting_Location_Surprisal,Level_of_Security_Surprisal,Leakage_Surprisal
Fame,1.000000,0.000943,0.327361,0.493260,0.630967,0.700094,0.858721,0.366029
Fame_av,0.000943,1.000000,0.498392,0.247493,0.509102,0.605901,0.720843,0.231784
Injured,0.327361,0.498392,1.000000,0.972935,0.660120,0.201070,0.075621,0.152220
Fatalities,0.493260,0.247493,0.972935,1.000000,0.114602,0.316234,0.413822,0.717212
Target_Group_Surprisal,0.630967,0.509102,0.660120,0.114602,1.000000,0.930999,0.927937,0.913435
Shooting_Location_Surprisal,0.700094,0.605901,0.201070,0.316234,0.930999,1.000000,0.439414,1.000000
Level_of_Security_Surprisal,0.858721,0.720843,0.075621,0.413822,0.927937,0.439414,1.000000,0.389209
Leakage_Surprisal,0.366029,0.231784,0.152220,0.717212,0.913435,1.000000,0.389209,1.000000


In [18]:
fame_seeking_df = Mass_shooting_df[(Mass_shooting_df["FameSeeker_Silva"]==True)]
Correlation_FS_df = fame_seeking_df.dropna(subset=['Fame'])[["Fame","Fame_av","Injured","Fatalities","Target_Group_Surprisal",
                                                          "Shooting_Location_Surprisal","Level_of_Security_Surprisal",
                                                          "Leakage_Surprisal"]]
display(Correlation_FS_df.corr(method="spearman")[["Injured","Fatalities"]])
display(Correlation_FS_df.corr(method=spearmanr_pval)[["Injured","Fatalities"]])

,Injured,Fatalities
Fame,0.948683,0.400000
Fame_av,0.948683,0.400000
Injured,1.000000,0.632456
Fatalities,0.632456,1.000000
Target_Group_Surprisal,-0.333333,-0.632456
Shooting_Location_Surprisal,-0.316228,0.400000
Level_of_Security_Surprisal,-0.316228,0.000000
Leakage_Surprisal,0.316228,0.000000


,Injured,Fatalities
Fame,0.051317,0.600000
Fame_av,0.051317,0.600000
Injured,1.000000,0.367544
Fatalities,0.367544,1.000000
Target_Group_Surprisal,0.666667,0.367544
Shooting_Location_Surprisal,0.683772,0.600000
Level_of_Security_Surprisal,0.683772,1.000000
Leakage_Surprisal,0.683772,1.000000


In [20]:
df_Target_group = pd.read_excel("Mass_Shootings_dummy.xlsx", sheet_name = "Alpha&Kappa_TargetGroup")
df_Target_group = df_Target_group.set_index('subject')
df_Target_group = df_Target_group.replace("School Community",3)
df_Target_group = df_Target_group.replace("Random",2)
df_Target_group = df_Target_group.replace("Past Interaction",1)
df_Target_group = df_Target_group.replace("Demographic Group",0)
Target_group = df_Target_group.to_numpy()
Target_group_Kappa = irr.fleiss_kappa(irr.aggregate_raters(Target_group)[0], method='rand')
Target_group_Alpha = cronbach_alpha(df_Target_group)

In [21]:
df_Level_of_Security = pd.read_excel("Mass_Shootings_dummy.xlsx", sheet_name = "Alpha&Kappa_LvlofSecurity")
df_Level_of_Security = df_Level_of_Security.set_index('subject')
df_Level_of_Security = df_Level_of_Security.replace("No-Security", 0)
df_Level_of_Security = df_Level_of_Security.replace("Low-Security", 1)
df_Level_of_Security = df_Level_of_Security.replace("High-Security", 2)
Level_of_Security = df_Level_of_Security.to_numpy()
Level_of_Security_Kappa = irr.fleiss_kappa(irr.aggregate_raters(Level_of_Security)[0], method='rand')
Level_of_Security_Alpha = cronbach_alpha(df_Level_of_Security)

In [22]:
print("Target Group Alpha: ", Target_group_Alpha)
print("Target Group Kappa: ", Target_group_Kappa)
print("Level of Security Alpha: ", Level_of_Security_Alpha)
print("Level of Security Kappa: ", Level_of_Security_Kappa)

Target Group Alpha:  0.9613477788273015
Target Group Kappa:  0.68
Level of Security Alpha:  0.9429097406218033
Level of Security Kappa:  0.7600000000000002


In [24]:
display(Mass_shooting_df[(Mass_shooting_df["FameSeeker_Silva"]==True)][["Target_Group_Surprisal","Shooting_Location_Surprisal","Level_of_Security_Surprisal","Leakage_Surprisal","Total Firearms Brought to the Scene"]].corr(method='spearman'))
display(Mass_shooting_df[(Mass_shooting_df["FameSeeker_Silva"]==True)][["Target_Group_Surprisal","Shooting_Location_Surprisal","Level_of_Security_Surprisal","Leakage_Surprisal","Total Firearms Brought to the Scene"]].corr(method=spearmanr_pval))

,Target_Group_Surprisal,Shooting_Location_Surprisal,Level_of_Security_Surprisal,Leakage_Surprisal,Total Firearms Brought to the Scene
Target_Group_Surprisal,1.000000,-0.737865,-0.737865,0.737865,0.948683
Shooting_Location_Surprisal,-0.737865,1.000000,0.800000,-0.800000,-0.600000
Level_of_Security_Surprisal,-0.737865,0.800000,1.000000,-1.000000,-0.800000
Leakage_Surprisal,0.737865,-0.800000,-1.000000,1.000000,0.800000
Total Firearms Brought to the Scene,0.948683,-0.600000,-0.800000,0.800000,1.000000


,Target_Group_Surprisal,Shooting_Location_Surprisal,Level_of_Security_Surprisal,Leakage_Surprisal,Total Firearms Brought to the Scene
Target_Group_Surprisal,1.000000,0.262135,0.262135,0.262135,0.051317
Shooting_Location_Surprisal,0.262135,1.000000,0.200000,0.200000,0.400000
Level_of_Security_Surprisal,0.262135,0.200000,1.000000,0.000000,0.200000
Leakage_Surprisal,0.262135,0.200000,0.000000,1.000000,0.200000
Total Firearms Brought to the Scene,0.051317,0.400000,0.200000,0.200000,1.000000


In [25]:
count = 0
for col in Wikipedia_df.columns:
    if col != "DateTime":
        arr = Wikipedia_df[col].values
        nonzeroarrTS = arr[np.where(arr != 0)[0][0]:]
        print(adfuller(nonzeroarrTS)[1])
        if adfuller(nonzeroarrTS)[1] > 0.05:
            count +=1
print(count)

0.0
5.4167899814727044e-20
0.0
6.741717043509473e-16
2.828670632253555e-23
4.5061028867105114e-12
0.0
2.2283441965692223e-07
2.0804671579393384e-06
0.0006037053086086647
0
